In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [2]:
data = pd.read_csv('dataset.csv')

In [3]:
data = data.sample(frac=1).reset_index(drop=True)

In [4]:
columns_titles = ["Tweet", "Label"]
data = data.reindex(columns=columns_titles)

In [5]:
data = pd.concat([data,pd.DataFrame(data['Label'])],axis=1)

In [6]:
train_data = data.head(int(225*0.8))

In [7]:
test_data = data.tail(int(225*0.2))

In [8]:
display(train_data)
display(test_data)

,Tweet,Label,Label
0,"""RT @dwnews: """"The government is useless. They...",1,1
1,RT @TheAtlantic: Growing numbers of people in ...,0,0
2,RT @annafifield: A person has died of coronavi...,1,1
3,RT @estevez_damiana: @DeZurdaTeam @DiazCanelB ...,0,0
4,RT @Dianacuba20191: @DeZurdaTeam @DiazCanelB @...,1,1
...,...,...,...
175,RT @balajis: The #coronavirus has now killed m...,0,0
176,RT @AnitaFernandex: @DeZurdaTeam @DiazCanelB @...,1,1
177,RT @KenRoth: One of the first Chinese doctors ...,0,0
178,RT @AJEnglish: Why anger over the Hong Kong go...,1,1


,Tweet,Label,Label
181,RT @vilmadiazcuba: @DeZurdaTeam @DiazCanelB @m...,0,0
182,RT @ABC: Hundreds of Hong Kong hospital worker...,0,0
183,RT @Independent: Hong Kong hospital workers st...,0,0
184,RT @ChannelNewsAsia: Hong Kong medical workers...,1,1
185,"RT @MailOnline: 2,400 medical workers go on st...",0,0
186,RT @alfonslopeztena: Hundreds of Hong Kong med...,0,0
187,RT @MarioFernndezV2: @DeZurdaTeam @DiazCanelB ...,1,1
188,RT @AmandadeCuba1: @DeZurdaTeam @DiazCanelB @m...,0,0
189,RT @frontlinepbs: As the number of cases of th...,0,0
190,@TheEconomist The latest coronavirus cases as ...,0,0
